In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


pd.set_option("display.max_columns", None)

# Dataset credit goes to
#### https://github.com/LuisM78/Appliances-energy-prediction-data
#### https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction

##### The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters).

| Variable Name | Role     | Type        | Description | Units  | Missing Values |
|----------------|----------|-------------|--------------|--------|----------------|
| date           | Feature  | Date        | Timestamp of observation | — | no |
| Appliances     | Target   | Integer     | Energy use in household appliances | Wh | no |
| lights         | Feature  | Integer     | Energy use of lights in the house | Wh | no |
| T1             | Feature  | Continuous  | Temperature in kitchen area | °C | no |
| RH_1           | Feature  | Continuous  | Humidity in kitchen area | % | no |
| T2             | Feature  | Continuous  | Temperature in living room | °C | no |
| RH_2           | Feature  | Continuous  | Humidity in living room | % | no |
| T3             | Feature  | Continuous  | Temperature in laundry room | °C | no |
| RH_3           | Feature  | Continuous  | Humidity in laundry room | % | no |
| T4             | Feature  | Continuous  | Temperature in office room | °C | no |
| RH_4           | Feature  | Continuous  | Humidity in office room | % | no |
| T5             | Feature  | Continuous  | Temperature in bathroom | °C | no |
| RH_5           | Feature  | Continuous  | Humidity in bathroom | % | no |
| T6             | Feature  | Continuous  | Temperature in north bedroom | °C | no |
| RH_6           | Feature  | Continuous  | Humidity in north bedroom | % | no |
| T7             | Feature  | Continuous  | Temperature in ironing room | °C | no |
| RH_7           | Feature  | Continuous  | Humidity in ironing room | % | no |
| T8             | Feature  | Continuous  | Temperature in teenager room | °C | no |
| RH_8           | Feature  | Continuous  | Humidity in teenager room | % | no |
| T9             | Feature  | Continuous  | Temperature in parents’ room | °C | no |
| RH_9           | Feature  | Continuous  | Humidity in parents’ room | % | no |
| T_out          | Feature  | Continuous  | Outdoor temperature | °C | no |
| Press_mm_hg    | Feature  | Continuous  | Outdoor pressure | mm Hg | no |
| RH_out         | Feature  | Continuous  | Outdoor humidity | % | no |
| Windspeed      | Feature  | Continuous  | Wind speed | m/s | no |
| Visibility     | Feature  | Continuous  | Outdoor visibility | km | no |
| Tdewpoint      | Feature  | Continuous  | Dew point temperature | °C | no |
| rv1            | Feature  | Continuous  | Random variable 1 (for testing purposes) | — | no |
| rv2            | Feature  | Continuous  | Random variable 2 (for testing purposes) | — | no |



# Load the Dataset

In [3]:
CSV_LOCATION = "Dataset/energydata_complete.csv"

df = pd.read_csv(CSV_LOCATION)
df.sample(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
308,2016-01-13 20:20:00,420,20,19.50,41.326667,19.0,40.500000,19.20,42.29,18.76,41.266667,18.30,78.600,1.633333,82.733333,18.50,38.156667,19.79,45.133333,16.73,40.2,3.133333,756.9,88.333333,5.0,32.666667,1.4,3.422332,3.422332
6774,2016-02-27 18:00:00,90,10,20.89,34.890000,19.5,33.466667,20.79,35.09,20.10,32.090000,17.89,44.356,5.126667,33.866667,20.79,26.468571,22.39,33.826667,18.10,39.7,4.800000,751.3,58.000000,5.0,29.000000,-2.8,42.328549,42.328549


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [5]:
df.dtypes

date            object
Appliances       int64
lights           int64
T1             float64
RH_1           float64
T2             float64
RH_2           float64
T3             float64
RH_3           float64
T4             float64
RH_4           float64
T5             float64
RH_5           float64
T6             float64
RH_6           float64
T7             float64
RH_7           float64
T8             float64
RH_8           float64
T9             float64
RH_9           float64
T_out          float64
Press_mm_hg    float64
RH_out         float64
Windspeed      float64
Visibility     float64
Tdewpoint      float64
rv1            float64
rv2            float64
dtype: object

In [6]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,19.592106,50.949283,7.910939,54.609083,20.267106,35.388200,22.029107,42.936165,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,1.844623,9.022034,6.090347,31.149806,2.109993,5.114208,1.956162,5.224361,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,15.330000,29.815000,-6.065000,1.000000,15.390000,23.200000,16.306667,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,18.277500,45.400000,3.626667,30.025000,18.700000,31.500000,20.790000,39.066667,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,19.390000,49.090000,7.300000,55.290000,20.033333,34.863333,22.100000,42.375000,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,20.619643,53.663333,11.256000,83.226667,21.600000,39.000000,23.390000,46.536000,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,25.795000,96.321667,28.290000,99.900000,26.000000,51.400000,27.230000,58.780000,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [7]:
# Datetime features

df["date"] = pd.to_datetime(df["date"])
df["hour"] = df["date"].dt.hour
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

In [8]:
df.sample(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,day_of_week,month,is_weekend
12944,2016-04-10 14:20:00,50,0,22.666667,38.09,23.133333,35.290000,22.6,38.663333,22.00,36.70,20.52,52.596,15.30,7.99,22.76,32.23,22.698571,36.238571,20.790000,39.700000,12.866667,752.4,58.666667,5.0,32.666667,4.9,3.716911,3.716911,14,6,4,1
1647,2016-01-23 03:30:00,40,0,17.390000,42.40,16.463333,42.663333,17.7,40.700000,15.39,41.79,15.60,48.700,5.59,99.90,15.80,37.29,16.790000,45.090000,15.226667,41.626667,5.050000,765.5,99.500000,3.0,2.500000,5.0,13.000455,13.000455,3,5,1,1


In [9]:
# cyclic encoding for hour

df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)


# Drop the raw date
df = df.drop(columns = ['date'])

In [10]:
target = "Appliances"
X = df.drop(columns = [target])
y = df[target]

# Time based split
split_idx = int(len(df) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

In [11]:
# Lets the sample size for each

print(f"\nTraining sample size: {len(X_train)}")
print(f"Test sample size: {len(X_test)}")


Training sample size: 15788
Test sample size: 3947


In [12]:
model = RandomForestRegressor(
    n_estimators = 300,
    max_depth = 10,
    random_state = 41,
    n_jobs = -1
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

mae = mean_absolute_error(y_test, y_pred),
r2 = r2_score(y_test, y_pred)

print(f"\nRMSE: {rmse:.2f}")
print(f"MAE: {mae[0]:.2f}")
print(f"R2 Score: {r2:.3f}")


RMSE: 204.52
MAE: 164.79
R2 Score: -4.047


In [13]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 5, 10, 15] 
}

model_grid = GridSearchCV(
    RandomForestRegressor(random_state=41),
    param_grid=param_grid,
    n_jobs = -1
)

model_grid.fit(X_train, y_train)

print("Best parameters:", model_grid.best_params_)
print("Best CV R2 Score:", model_grid.best_score_)


# Evaluate tuned model
y_pred_tuned = model_grid.predict(X_test)
print(f"\nTuned Random Forest R2 on Test: {r2_score(y_test, y_pred_tuned):.4f}")

Best parameters: {'max_depth': 5, 'n_estimators': 50}
Best CV R2 Score: 0.11809846438547249

Tuned Random Forest R2 on Test: -1.6052


In [14]:
# Now lets move to XGBoost

from xgboost import XGBRegressor

model_xgboost = XGBRegressor(
    n_estimators = 15,
    learning_rate = 0.01
)

model_xgboost.fit(X_train, y_train)

y_pred = model_xgboost.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R2 Score: {r2:.3f}")

RMSE: 88.81
MAE: 55.39
R2 Score: 0.048


In [15]:
# # Hyper tune this xg boost model

# param_grid_xgboost = {
#     "n_estimators" : [100, 200, 500],
#     "learning_rate" : [0.01, 0.05, 0.09, 0.1],
#     'max_depth' : [3, 5, 7 ,10],
#     'subsample' : [0.8, 0.9, 1.0]
# }

# Grid_model_xgboost = GridSearchCV(
#     estimator=XGBRegressor(),
#     param_grid=param_grid_xgboost,
#     n_jobs = -1
# )

# Grid_model_xgboost.fit(X_train, y_train)

# print("Best parameters:", Grid_model_xgboost.best_params_)
# print("Best CV R2 Score:", Grid_model_xgboost.best_score_)


# # Evaluate tuned model
# y_pred_tuned = Grid_model_xgboost.predict(X_test)
# print(f"\nTuned Random Forest R2 on Test: {r2_score(y_test, y_pred_tuned):.4f}")



In [16]:
# mse = mean_squared_error(y_test, y_pred_tuned)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred_tuned)
# r2 = r2_score(y_test, y_pred_tuned)

# print(f"RMSE: {rmse:.2f}")
# print(f"MAE: {mae:.2f}")
# print(f"R2 Score: {r2:.3f}")

In [17]:
# sanity check

xgb_simple = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_simple.fit(X_train, y_train)
y_pred_simple = xgb_simple.predict(X_test)

mse = mean_squared_error(y_test, y_pred_simple)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred_simple)
r2 = r2_score(y_test, y_pred_simple)

print(f"Simple XGB -> RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.3f}")


Simple XGB -> RMSE: 207.76, MAE: 175.52, R2: -4.208


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # --- 1) Ensure sorted by time ---
# df["date"] = pd.to_datetime(df["date"])
# df = df.sort_values("date").reset_index(drop=True)

# # --- 2) Datetime features ---
# df["hour"] = df["date"].dt.hour
# df["day_of_week"] = df["date"].dt.dayofweek
# df["month"] = df["date"].dt.month
# df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

# # Cyclic encoding for hour
# df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
# df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

# --- 3) Lag & rolling features for Appliances (key improvement) ---
# 10‑min data: 6 steps = 1 hour, 3 steps = 30 min
df["Appliances_lag1"] = df["Appliances"].shift(1)
df["Appliances_lag2"] = df["Appliances"].shift(2)
df["Appliances_lag6"] = df["Appliances"].shift(6)

df["Appliances_roll3_mean"] = df["Appliances"].rolling(window=3).mean()
df["Appliances_roll6_mean"] = df["Appliances"].rolling(window=6).mean()

# Drop first rows with NaNs from lags/rollings
df = df.dropna().reset_index(drop=True)

# --- 4) Drop known noise / low‑value columns ---
# rv1 and rv2 are documented random variables
df = df.drop(columns=["rv1", "rv2"])

# Drop raw date if you don't want it as a feature
# df = df.drop(columns=["date"])

# --- 5) Define X, y ---
target = "Appliances"
X = df.drop(columns=[target])
y = df[target]

# --- 6) Time‑based split (no shuffling) ---
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Train shape: (15783, 36) Test shape: (3946, 36)


In [25]:
model = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,           # let trees grow deeper; you can tune later
    min_samples_leaf=5,       # helps generalization
    max_features="sqrt",
    random_state=41,
    n_jobs=-1
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nRMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R2 Score: {r2:.3f}")



RMSE: 39.44
MAE: 22.22
R2 Score: 0.812


In [14]:
from xgboost import XGBRegressor

# sanity check

xgb_simple = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_simple.fit(X_train, y_train)
y_pred_simple = xgb_simple.predict(X_test)

mse = mean_squared_error(y_test, y_pred_simple)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred_simple)
r2 = r2_score(y_test, y_pred_simple)

print(f"Simple XGB -> RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.3f}")


Simple XGB -> RMSE: 12.94, MAE: 6.67, R2: 0.980
